# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1: José Jesús La Casa Nieto
*   Alumno 2:
*   Alumno 3:






---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.


---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```


---
### 1.2. Localizar entorno de trabajo: Google colab o local

In [1]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [2]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

Archivos en el directorio: 
['checkpoint', 'dqn2_SpaceInvaders-v0_log.json', 'dqn2_SpaceInvaders-v0_weights.h5f.data-00000-of-00001', 'dqn2_SpaceInvaders-v0_weights.h5f.index', 'dqn2_SpaceInvaders-v0_weights_250000.h5f.data-00000-of-00001', 'dqn2_SpaceInvaders-v0_weights_250000.h5f.index', 'dqn2_SpaceInvaders-v0_weights_500000.h5f.data-00000-of-00001', 'dqn2_SpaceInvaders-v0_weights_500000.h5f.index', 'dqn3_SpaceInvaders-v0_log.json', 'dqn3_SpaceInvaders-v0_weights.h5f.data-00000-of-00001', 'dqn3_SpaceInvaders-v0_weights.h5f.index', 'dqn3_SpaceInvaders-v0_weights_250000.h5f.data-00000-of-00001', 'dqn3_SpaceInvaders-v0_weights_250000.h5f.index', 'dqn3_SpaceInvaders-v0_weights_500000.h5f.data-00000-of-00001', 'dqn3_SpaceInvaders-v0_weights_500000.h5f.index', 'dqn4_SpaceInvaders-v0_weights.h5f.data-00000-of-00001', 'dqn4_SpaceInvaders-v0_weights.h5f.index', 'dqn_SpaceInvaders-v0_log.json', 'dqn_SpaceInvaders-v0_weights.h5f.data-00000-of-00001', 'dqn_SpaceInvaders-v0_weights.h5f.index', 'd

---
### 1.4. Instalar librerías necesarias

In [ ]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.8
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.2.4
  %pip install tensorflow==2.5.3
  %pip install torch==2.0.1
  %pip install agents==1.4.0

---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [3]:
from __future__ import division

from PIL import Image
import numpy as np
import gym
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

#### Configuración base

In [4]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [5]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

In [6]:
print("Número de acciones disponibles: " + str(nb_actions))

Número de acciones disponibles: 6


In [7]:
print("Formato de observaciones:", env.observation_space)

Formato de observaciones: Box(0, 255, (210, 160, 3), uint8)


1. Implementación de la red neuronal

In [8]:
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()
print(K.image_data_format())

if K.image_data_format() == 'channels_last':
    model.add(Permute((2, 3, 1), input_shape=input_shape))
elif K.image_data_format() == 'channels_first':
    model.add(Permute((1, 2, 3), input_shape=input_shape))
else:
    raise RuntimeError('Unknown image_dim_ordering.')

model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

channels_last
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute (Permute)            (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 20, 20, 32)        8224      
_________________________________________________________________
activation (Activation)      (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
activation_1 (Activation)    (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
activation_2 (Activation)    (None, 7, 7, 

2. Implementación de la solución DQN

In [9]:
memory = SequentialMemory(limit=100000, window_length=WINDOW_LENGTH)    # Reduzco un poco la memoria
processor = AtariProcessor()

In [10]:
policy = LinearAnnealedPolicy(
    EpsGreedyQPolicy(),
    attr='eps',
    value_max=1.,
    value_min=.1,
    value_test=.2,
    nb_steps=100000
)

In [11]:
dqn = DQNAgent(
    model=model,
    nb_actions=nb_actions,
    policy=policy,
    memory=memory,
    processor=processor,
    nb_steps_warmup=100000,   # Aumento el número de pasos iniciales porque el entorno es complejo y así obtiene una base sólida de experiencias
    gamma=.99,
    target_model_update=100000,  # Mantengo las mismas actualizaciones de las redes. Valores altos suponen mayor estabilidad pero más tiempo de entrenamiento
    train_interval=4,
    enable_dueling_network=True,
    dueling_type='avg'
)
dqn.compile(Adam(learning_rate=.1e-4), metrics=['mae'])     # Cambio en el learning rate

In [12]:
# Train
weights_filename = 'dqn5_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn5_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn5_{}_log.json'.format(env_name)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]

dqn.fit(
    env,
    callbacks=callbacks,
    nb_steps=1000000,
    log_interval=1000,
    visualize=False
)

dqn.save_weights(weights_filename, overwrite=True)

Training for 1000000 steps ...
Interval 1 (0 steps performed)


c:\Users\pepel\.conda\envs\Miar2_rl38\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


1000/1000 [==============================] - 6s 3ms/step - reward: 0.0120
2 episodes - episode_reward: 6.000 [6.000, 6.000] - ale.lives: 2.233

Interval 2 (1000 steps performed)
1000/1000 [==============================] - 3s 3ms/step - reward: 0.0150
2 episodes - episode_reward: 7.000 [6.000, 8.000] - ale.lives: 2.224

Interval 3 (2000 steps performed)
1000/1000 [==============================] - 3s 3ms/step - reward: 0.0150
1 episodes - episode_reward: 13.000 [13.000, 13.000] - ale.lives: 2.332

Interval 4 (3000 steps performed)
1000/1000 [==============================] - 3s 3ms/step - reward: 0.0170
1 episodes - episode_reward: 6.000 [6.000, 6.000] - ale.lives: 1.892

Interval 5 (4000 steps performed)
1000/1000 [==============================] - 3s 3ms/step - reward: 0.0220
1 episodes - episode_reward: 26.000 [26.000, 26.000] - ale.lives: 1.856

Interval 6 (5000 steps performed)
1000/1000 [==============================] - 3s 3ms/step - reward: 0.0130
1 episodes - episode_reward: 1

In [31]:
# Testing part to calculate the mean reward
weights_filename = 'dqn5_{}_weights.h5f'.format(env_name)
dqn.load_weights(weights_filename)
results = dqn.test(env, nb_episodes=10, visualize=True)

Testing for 10 episodes ...
Episode 1: reward: 28.000, steps: 1024
Episode 2: reward: 19.000, steps: 590
Episode 3: reward: 27.000, steps: 854
Episode 4: reward: 22.000, steps: 830
Episode 5: reward: 12.000, steps: 480
Episode 6: reward: 32.000, steps: 1313
Episode 7: reward: 17.000, steps: 604
Episode 8: reward: 31.000, steps: 1157
Episode 9: reward: 33.000, steps: 1089
Episode 10: reward: 23.000, steps: 839


In [32]:
print(f"Average reward: {np.mean(results.history['episode_reward']):.1f}")

Average reward: 24.4


3. Justificación de los parámetros seleccionados y de los resultados obtenidos

# Cronología

### Primeros pasos

Iniciamos el proyecto práctico utilizando como base la sesión práctica 5 de la asignatura, a través del notebook por defecto de la sesión. Creamos nuestro propio entorno virtual en local y ejecutamos el modelo con una red neuronal similar al notebook de la sesión práctica. El promedio de las recompensas obtenido oscilaba entre los 14 y 15 puntos de media, lo que nos llevó a iniciar una serie de cambios para mejorar estos resultados.

### Modificaciones iniciales

Las primeras modificaciones consistieron en aumentar los pasos de entrenamiento y cambiar el 'learning rate', acorde a lo aprendido en la asignatura de Redes Neuronales. Aumentar los pasos de entrenamiento nos permitió proporcionar más datos y tiempo al modelo para aprender, mientras que ajustar la tasa de aprendizaje nos ayudó a controlar la velocidad a la que el modelo se ajustaba a los datos. Sin embargo, estos ajustes iniciales no fueron suficientes, por lo que decidimos hacer más compleja la arquitectura de la red neuronal.

### Primera modificación de la red neuronal

Empezamos añadiendo capas convolucionales, incrementando la profundidad del modelo y añadiendo normalizaciones. De esta manera, buscábamos poder recabar más información de cada una de las imágenes que fuesen procesadas por la red neuronal, procedentes de las capturas de pantalla del videojuego.

| Layer (type)                               | Output Shape       | Param # |
|--------------------------------------------|--------------------|---------|
| permute_1 (Permute)                        | (None, 84, 84, 4)  | 0       |
| conv2d_3 (Conv2D)                          | (None, 21, 21, 32) | 8224    |
| activation_5 (Activation)                  | (None, 21, 21, 32) | 0       |
| batch_normalization_4 (BatchNormalization) | (None, 21, 21, 32) | 128     |
| dropout_4 (Dropout)                        | (None, 21, 21, 32) | 0       |
| conv2d_4 (Conv2D)                          | (None, 11, 11, 64) | 32832   |
| activation_6 (Activation)                  | (None, 11, 11, 64) | 0       |
| batch_normalization_5 (BatchNormalization) | (None, 11, 11, 64) | 256     |
| dropout_5 (Dropout)                        | (None, 11, 11, 64) | 0       |
| conv2d_5 (Conv2D)                          | (None, 11, 11, 64) | 36928   |
| activation_7 (Activation)                  | (None, 11, 11, 64) | 0       |
| batch_normalization_6 (BatchNormalization) | (None, 11, 11, 64) | 256     |
| dropout_6 (Dropout)                        | (None, 11, 11, 64) | 0       |
| flatten_1 (Flatten)                        | (None, 7744)       | 0       |
| dense_2 (Dense)                            | (None, 512)        | 3965440 |
| activation_8 (Activation)                  | (None, 512)        | 0       |
| batch_normalization_7 (BatchNormalization) | (None, 512)        | 2048    |
| dropout_7 (Dropout)                        | (None, 512)        | 0       |
| dense_3 (Dense)                            | (None, 6)          | 3078    |
| activation_9 (Activation)                  | (None, 6)          | 0       |

**Total params:** 4,049,190  
**Trainable params:** 4,047,846  
**Non-trainable params:** 1,344

Expandimos la arquitectura a tres capas convolucionales, con la intención de mejorar aún más la capacidad del modelo para aprender representaciones significativas. Las normalizaciones se añadieron para evitar el sobreentrenamiento y que la red aprendiera de una manera más uniforme.

En cuanto al agente, la configuración se mantuvo con respecto a los intentos anteriores. Al no obtener mejoras significativas, decidimos aumentar los steps a 1750000, pero los resultados no tuvieron la mejora que se esperaba, siendo la media inferior a 17.

![Media de recompensas con 10 episodios de 16.9](Images/Score_17.png)

### Tratamiento de la 'policy'

El siguiente parámetro que decidimos cambiar y utilizar fue la 'policy'. Teniendo en cuenta que el agente es invariable en nuestra práctica (DQN), probamos con 'Boltzmann' y 'DecayEpsGreedy'. Sin embargo, los resultados no fueron significativos, llegando a empeorar los resultados previos a las pruebas.

### Memoria y 'steps_warmup'

A continuación, aumentamos el parámetro del agente DQN `steps_warmup` y el límite de la memoria de experiencia. La intención de estos cambios es aumentar el almacenamiento con la que el modelo guarda las experiencias vividas durante la exploración. Es por ello, que el aumento de 'steps_warmup' tiene sentido para que el modelo explore y aprenda de situaciones diferentes, obteniendo así una base de conocimiento sobre la que entrenará en los siguientes pasos. Esta estrategia de aumento de la memoria y de los pasos iniciales de exploración, es ideal para problemas como los que nos enfrentamos en el proyecto, donde el juego tiene cierta complejidad y necesita aprender del entorno.

Este cambio asumido por el grupo en el proyecto, permitió un ligero aumento de los resultados obtenidos por el modelo. Ahora sí, conseguimos alcanzar el mínimo de 20 puntos de recompensa de media para 10 partidas. Sin embargo, el modelo no siempre obtiene esa media deseada, puesto que la varianza de los resultados es ciertamente grande y el modelo se entrena con demasiados parámetros (un entrenamiento cualquiera podía llegar a muchas horas sin obtener mejoras significativas por la red neuronal).

![Media de recompensas con 10 episodios de 21.4](Images/Score_18.png)

![Media de recompensas con 10 episodios de 21.4](Images/Score_21.png)

Como caso curioso durante el entrenamiento y pruebas de distintos parámetros, se decidió entrenar un agente con poca memoria y un `steps_warmup` para la exploración bastante bajos. Para nuestra sorpresa, obtuvimos un modelo que proporcionaba buenos resultados en los tests de evaluación. Sin embargo, al observar las partidas jugadas, se aprecia que el agente no se mueve prácticamente de su posición, dedicándose exclusivamente a disparar.

<p align="center">
  <img src="Images/Juego_1.png" alt="Imagen del juego 1" />
</p>

<p align="center">
  <img src="Images/Juego_2.png" alt="Imagen del juego 2" />
</p>

<p align="center">
  <img src="Images/Juego_3.png" alt="Imagen del juego 3" />
</p>

![Media de recompensas con 10 episodios de 20.0](Images/Score_20.png)

La conclusión que obtenemos en este caso, es que unos buenos resultados de recompensa no son motivo para justificar que un modelo sea realmente bueno. Lo que sucedía en estas partidas fue que el modelo conseguía eliminar a gran parte de los enemigos, suponiendo un valor de recompensa alto, pero todos esos enemigos se encontraban en la zona izquierda de la pantalla porque el agente no se movía de su posición. Además, todas las partidas eran prácticamente iguales, por lo que nuestro agente no es capaz de realizar distintas estrategias para abordar el mismo problema.

### Simplificación de la red

En base a la explicación anterior, decidimos simplificar la red neuronal con la idea de optimizar el proceso de entrenamiento y quizás, obtener mejores resultados que los anteriores.

| Layer (type)                 | Output Shape         | Param #   |
|------------------------------|----------------------|-----------|
| permute (Permute)            | (None, 84, 84, 4)    | 0         |
| conv2d (Conv2D)              | (None, 20, 20, 32)   | 8224      |
| activation (Activation)      | (None, 20, 20, 32)   | 0         |
| conv2d_1 (Conv2D)            | (None, 9, 9, 64)     | 32832     |
| activation_1 (Activation)    | (None, 9, 9, 64)     | 0         |
| conv2d_2 (Conv2D)            | (None, 7, 7, 64)     | 36928     |
| activation_2 (Activation)    | (None, 7, 7, 64)     | 0         |
| flatten (Flatten)            | (None, 3136)         | 0         |
| dense (Dense)                | (None, 512)          | 1606144   |
| activation_3 (Activation)    | (None, 512)          | 0         |
| dense_1 (Dense)              | (None, 256)          | 131328    |
| activation_4 (Activation)    | (None, 256)          | 0         |
| dense_2 (Dense)              | (None, 6)            | 1542      |
| activation_5 (Activation)    | (None, 6)            | 0         |

**Total params:** 1,816,998  
**Trainable params:** 1,816,998  
**Non-trainable params:** 0

Este cambio, supuso una reducción de los parámetros entrenables en más de un 50%. Lo más importante, los resultados medios de recompensa seguían siendo prácticamente iguales a los de la red neuronal anterior.

### Resultados finales:

Finalmente, se decidió conservar la estrategia de los parámetros del agente DQN, utilizar `LinearAnnealedPolicy` como 'policy' y utilizar la red neuronal optimizada. De esta manera, y con una estrategia más o menos acertada, entrenamos el modelo por un gran número de pasos, con la idea de mejorar los resultados lo máximo posible. Se utilizaron 1 millón de pasos de entrenamiento (como se puede ver en los apartados anteriores de este notebook), con una duración de entrenamiento de más de 7 horas.

| **Episode** | **Reward** | **Steps** |
|-------------|------------|-----------|
| Episode 1   | 28.000     | 1024      |
| Episode 2   | 19.000     | 590       |
| Episode 3   | 27.000     | 854       |
| Episode 4   | 22.000     | 830       |
| Episode 5   | 12.000     | 480       |
| Episode 6   | 32.000     | 1313      |
| Episode 7   | 17.000     | 604       |
| Episode 8   | 31.000     | 1157      |
| Episode 9   | 33.000     | 1089      |
| Episode 10  | 23.000     | 839       |

=======================

**Average reward: 24.4**

La recompensa media llegó a ser de 24.4. Además, obtuvimos resultados de partidas con un 'reward' de 31 y 33. En esas partidas en concreto, el modelo casi consigue finalizar la partida al completo eliminando a casi todas las naves rivales.

![Media de recompensas con 10 episodios de 24.4](Images/Score_24.png)

# Parámetros controlables durante el proyecto

En este apartado, queremos explicar con más detenimiento los parámetros que hemos ido probando a lo largo de la práctica y cómo pueden afectar al rendimiento del modelo durante las pruebas de evaluación.

### Implementación de la Red Neuronal

La red neuronal es el "cerebro" de nuestro agente DQN. La arquitectura de red neuronal que hemos utilizado dispone de tres capas convolucionales (CNNs). Las redes neuronales convolucionales son especialmente eficaces para procesar imágenes, lo que es clave para nuestro agente en este proyecto, que debe interpretar y tomar decisiones basadas en la observación del estado del juego.

1. La red neuronal recibe como entrada un conjunto de 4 cuadros de 84x84 píxeles (ventana temporal de 4), lo cual permite que el agente tenga en cuenta la temporalidad de los movimientos. Esto es clave para que el modelo entienda hacia dónde se mueven los disparos o el desplazamiento de los enemigos. Utilizamos un procesador personalizado (AtariProcessor) que convierte las imágenes en escala de grises y redimensiona las observaciones para resaltar las características relevantes.

2. Capas convolucionales:

- **Primera capa**: 32 filtros de 8x8 con strides de 4, seguidos de una activación ReLU. Esta capa captura características espaciales de bajo nivel como bordes y esquinas.
- **Segunda capa**: 64 filtros de 4x4 con strides de 2, con activación ReLU, para capturar patrones más complejos.
- **Tercera capa**: 64 filtros de 3x3 con activación ReLU para detectar características aún más abstractas.

3. Capas densas:

- **Primera capa densa**: 512 unidades con activación ReLU para integrar las características extraídas por las capas convolucionales.
- **Segunda capa densa**: 256 unidades con activación ReLU para una mayor abstracción de la información.
- **Capa de salida**: Dispone de tantas neuronas como acciones posibles hay en el juego, con activación lineal para predecir los valores Q de cada acción.

Esta arquitectura permite que el agente capture información espacial y temporal del entorno, facilitando una toma de decisiones eficiente.

### Componentes de la Solución DQN

Tal y como se ha explicado en la cronología, la implementación de los parámetros del agente DQN es esencial para la mejora de los resultados. Aquí detallamos los componentes clave:

1. Memoria secuencial: Utilizamos una memoria secuencial (SequentialMemory) con un límite de 100,000 transiciones. Esto permite almacenar experiencias pasadas y muestrearlas durante el entrenamiento, mejorando la estabilidad del entrenamiento.

2. Política de exploración: Empleamos una política `LinearAnnealedPolicy` con `EpsGreedyQPolicy` para gestionar la correlación entre exploración y explotación, conceptos clave en cualquier entrenamiento de modelos de Aprendizaje por Refuerzo Profundo. La política comienza con una alta tasa de exploración (eps=1.0) que disminuye gradualmente a 0.1 durante 100.000 pasos, promoviendo la exploración inicial y la explotación en etapas posteriores.

3. Configuración del agente DQN:

- **nb_steps_warmup** de 100,000 pasos para una base sólida de experiencias antes de comenzar la actualización y entrenamiento de los pesos de la red.
- **gamma** de 0.99 para valorar las recompensas futuras.
- **target_model_update** cada 10,000 pasos para mayor estabilidad en las actualizaciones del modelo.
- **train_interval** de 4 pasos para actualizar la red neuronal de manera eficiente sin sobrecargar el sistema.

4. Habilitación de la red de duelo: Activar `dueling_network` con tipo 'avg' mejora la capacidad del agente para evaluar el valor de cada estado de forma más precisa, diferenciando entre el valor de estar en un estado y el valor de realizar una acción en dicho estado.

Estos componentes permiten que el agente aprenda de manera eficiente y estable en nuestro entorno del proyecto 'Space Invaders'.


# Bibliografía

Como material adicional a las clases del curso, hemos buscado información relevante en los siguientes enlaces:

- [Ventajas y desventajas del Aprendizaje por Refuerzo Profundo](https://huggingface.co/learn/deep-rl-course/unit4/advantages-disadvantages)
- [Métodos basados en políticas](https://huggingface.co/learn/deep-rl-course/unit4/what-are-policy-based-methods)
- [Atari Game en Kaggle](https://www.kaggle.com/code/elsall/atari-game)
- [Mastering Atari Games: Deep Reinforcement Learning Unleashed](https://medium.com/@chemistry8526/mastering-atari-games-deep-reinforcement-learning-unleashed-f16af5e2144a)
- [Space Invaders en Gymnasium](https://gymnasium.farama.org/environments/atari/space_invaders/)

---